In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import train_test_split

In [ ]:
def shrink_df(df):
    df.user_id = df.user_id.astype(np.uint16)
    df.movie_id = df.movie_id.astype(np.uint16)
    df.rating = df.rating.astype(np.uint8)
    
    return df

In [ ]:
df = pd.read_csv("data/user_data/ml_100k_user_data.csv")

In [ ]:
y = df.rating
X = df.drop(["rating"], axis = 1)

In [ ]:
del df

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=30)

In [ ]:
X_train['rating'] = y_train
X_test['rating'] = y_test

In [ ]:
del y_train
del y_test

In [ ]:
#X_train.to_csv("data/new_pp/udata_train_100k.csv", index=False)

In [ ]:
#X_test.to_csv("data/new_pp/udata_test_100k.csv", index=False)

In [ ]:
# read from the split csv

In [ ]:
df_train = pd.read_csv("data/new_pp/udata_train_100k.csv")

In [2]:
df_test = pd.read_csv("data/new_pp/udata_test_100k.csv")

In [ ]:
one_hot_train = pd.get_dummies(df_train.movie_id, prefix="movie_id", dtype=np.uint8)

In [ ]:
one_hot_train["user_id"] = df_train.user_id

In [ ]:
# reorder and sort the columns, write to csv, delete variables
one_hot_train = one_hot_train[one_hot_train.columns.tolist()[-1:] + one_hot_train.columns.tolist()[:-1]]

In [ ]:
group_train = one_hot_train.groupby("user_id")

In [ ]:
exposure_train = pd.DataFrame(columns=one_hot_train.columns)

In [ ]:
for i in tqdm(one_hot_train.user_id.unique().tolist()):
    temp = pd.DataFrame(group_train.get_group(i).drop("user_id", axis=1).sum()).T
    temp["user_id"] = i
    temp = temp[temp.columns.tolist()[-1:] + temp.columns.tolist()[:-1]]
    exposure_train = pd.concat([exposure_train, temp], ignore_index=True)

In [ ]:
exposure_train.shape

In [ ]:
# exposure_train.to_csv("data/new_pp/exposure_100k_train.csv", index=False)

In [3]:
train_df = pd.read_csv("data/new_pp/exposure_100k_train.csv")

In [4]:
train_df.shape

(943, 1659)

In [5]:
train_cols = train_df.columns.tolist()[1:]

In [6]:
del train_df

In [7]:
one_hot_test = pd.get_dummies(df_test.movie_id, prefix="movie_id", dtype=np.uint8)

In [8]:
one_hot_test["user_id"] = df_test.user_id

In [9]:
del df_test

In [10]:
one_hot_test.shape

(20000, 1387)

In [11]:
one_hot_test.head()

,movie_id_1,movie_id_2,movie_id_3,movie_id_4,movie_id_5,movie_id_6,movie_id_7,movie_id_8,movie_id_9,movie_id_10,...,movie_id_1651,movie_id_1668,movie_id_1669,movie_id_1671,movie_id_1672,movie_id_1674,movie_id_1677,movie_id_1679,movie_id_1681,user_id
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,758
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,358
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,772
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,683
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,312


In [12]:
train_cols = set(train_cols)
test_cols = set(one_hot_test.columns[:-1])

In [13]:
col_to_drop = list(test_cols - train_cols.intersection(test_cols))

In [14]:
col_to_drop[0]

'movie_id_1640'

In [15]:
one_hot_test = one_hot_test.drop(col_to_drop, axis=1)

In [16]:
one_hot_test.shape

(20000, 1363)

In [17]:
one_hot_test.head()

,movie_id_1,movie_id_2,movie_id_3,movie_id_4,movie_id_5,movie_id_6,movie_id_7,movie_id_8,movie_id_9,movie_id_10,...,movie_id_1610,movie_id_1611,movie_id_1612,movie_id_1615,movie_id_1617,movie_id_1622,movie_id_1628,movie_id_1643,movie_id_1672,user_id
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,758
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,358
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,772
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,683
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,312


In [18]:
col_to_add = list(train_cols - train_cols.intersection(test_cols))

In [20]:
col_to_add[0]

'movie_id_992'

In [21]:
one_hot_test.shape

(20000, 1363)

In [22]:
data = np.zeros((one_hot_test.shape[0],len(col_to_add)), dtype=np.uint8)

temp_df = pd.DataFrame(data, columns=col_to_add)

In [24]:
temp_df.head()

,movie_id_992,movie_id_1552,movie_id_1655,movie_id_1445,movie_id_1678,movie_id_1268,movie_id_1574,movie_id_1492,movie_id_788,movie_id_1438,...,movie_id_1586,movie_id_1191,movie_id_677,movie_id_1509,movie_id_1613,movie_id_1569,movie_id_1631,movie_id_1305,movie_id_1641,movie_id_1565
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [29]:
one_hot_test_n = pd.concat([one_hot_test, temp_df], axis=1)

In [30]:
one_hot_test_n.shape

(20000, 1659)

In [44]:
user_col = one_hot_test_n.user_id

In [45]:
one_hot_test_n = one_hot_test_n.drop(["user_id"], axis=1)

In [33]:
import re

def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    '''
    alist.sort(key=natural_keys) sorts in human order
    http://nedbatchelder.com/blog/200712/human_sorting.html
    (See Toothy's implementation in the comments)
    '''
    return [ atoi(c) for c in re.split(r'(\d+)', text) ]


In [46]:
col = one_hot_test_n.columns.tolist()
col.sort(key=natural_keys)

In [49]:
one_hot_test_n["user_id"] = user_col

In [50]:
one_hot_test_n = one_hot_test_n[one_hot_test_n.columns.tolist()[-1:] + one_hot_test_n.columns.tolist()[:-1]]

In [51]:
one_hot_test_n.shape

(20000, 1659)

In [52]:
one_hot_test_n.head()

,user_id,movie_id_1,movie_id_2,movie_id_3,movie_id_4,movie_id_5,movie_id_6,movie_id_7,movie_id_8,movie_id_9,...,movie_id_1586,movie_id_1191,movie_id_677,movie_id_1509,movie_id_1613,movie_id_1569,movie_id_1631,movie_id_1305,movie_id_1641,movie_id_1565
0,758,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,358,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,772,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,683,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,312,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [53]:
grouped = one_hot_test_n.groupby("user_id")

In [54]:
exposure_test = pd.DataFrame(columns=one_hot_test_n.columns)

In [55]:
for i in tqdm(one_hot_test_n.user_id.unique().tolist()):
    temp = pd.DataFrame(grouped.get_group(i).drop("user_id", axis=1).sum()).T
    temp["user_id"] = i
    temp = temp[temp.columns.tolist()[-1:] + temp.columns.tolist()[:-1]]
    exposure_test = pd.concat([exposure_test, temp], ignore_index=True)

100%|██████████| 943/943 [00:12<00:00, 76.12it/s] 


In [56]:
exposure_test.shape

(943, 1659)

In [57]:
exposure_test.head()

,user_id,movie_id_1,movie_id_2,movie_id_3,movie_id_4,movie_id_5,movie_id_6,movie_id_7,movie_id_8,movie_id_9,...,movie_id_1586,movie_id_1191,movie_id_677,movie_id_1509,movie_id_1613,movie_id_1569,movie_id_1631,movie_id_1305,movie_id_1641,movie_id_1565
0,758,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,358,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,772,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,683,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,312,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [58]:
exposure_test.to_csv("data/new_pp/exposure_100k_test.csv", index=False)